CODIGO PARA PARA ENTRENAR UN MODELO CON MÚLTIPLES IMÁGENES

##Codigo no probado por falta de imágenes##

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import pytesseract

In [ ]:
def cargar_datos(data_dir, img_size=(224, 224)):
    """
    Carga imágenes desde un directorio y genera X (features) y y (labels).
    """
    X = []  # Lista para almacenar las imágenes
    y = []  # Lista para almacenar las etiquetas
    clases = os.listdir(data_dir)  # Listar las clases en el directorio de datos
    for clase_id, clase in enumerate(clases):
        clase_path = os.path.join(data_dir, clase)  # Ruta del directorio de cada clase
        for img_name in os.listdir(clase_path):  # Listar las imágenes de cada clase
            img_path = os.path.join(clase_path, img_name)  # Ruta completa de la imagen
            img = cv2.imread(img_path)  # Leer la imagen con OpenCV
            if img is not None:  # Verificar que la imagen se haya cargado correctamente
                img_resized = cv2.resize(img, img_size)  # Redimensionar la imagen
                X.append(img_resized)  # Añadir la imagen redimensionada a X
                y.append(clase_id)  # Añadir la etiqueta correspondiente a y
    return np.array(X), np.array(y)  # Devolver las imágenes y las etiquetas como arrays de NumPy



In [ ]:
# Ruta al directorio de datos (ajusta la ruta según tu estructura)
train_data_dir = 'imagenes'  # Estructura: imagenes/clase_0/, imagenes/clase_1/

# Cargar datos
X, y = cargar_datos(train_data_dir)

# Normalizar X (escala 0-1)
X = X / 255.0

# Dividir en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
def build_model():
    """
    Construye y compila un modelo CNN.
    """
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')  # Clasificación binaria
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# Cargar y entrenar el modelo
model = build_model()
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

# Guardar el modelo entrenado
model.save("models/modelo_entrenado.h5")


In [ ]:
# Configuración de Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def procesar_imagen(image_path):
    """
    Carga y preprocesa una imagen.
    """
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray


In [ ]:
def inferencia(model, image_path):
    """
    Realiza la inferencia del modelo y Tesseract OCR.
    """
    image = procesar_imagen(image_path)
    
    # Realizar OCR con Tesseract
    ocr_result = pytesseract.image_to_string(image, config="--psm 6 -l eng")
    print("Resultado OCR:", ocr_result)

    # Preprocesar imagen para la red neuronal
    resized = cv2.resize(image, (224, 224)) / 255.0
    prediction = model.predict(resized.reshape(1, 224, 224, 1))
    print("Resultado Modelo CNN:", prediction)


In [ ]:
# Cargar el modelo guardado
model = tf.keras.models.load_model("models/modelo_entrenado.h5")

# Realizar inferencia en una imagen de prueba
inferencia(model, "data/imagen_prueba.jpg")
